# <font color=blue>   Análise Exploratória </font>
## <font color=black>   Menções ENE e CIC </font>

#### <font color=black>  Estudo sobre as disciplinas dos departamento de Engenharia Elétrica (ENE) e departamento de computação (CIC).</font>
#### <font color=black> Escopo de análise delimitado entre as disciplinas de graduação do primeiro semestre de 2009 até o último semestre de 2018 de ambos os departamentos.</font>

In [2]:
#importar bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import plotly
import plotly.graph_objs as go
import plotly.plotly as py
import cufflinks
import warnings
from plotly.offline import iplot, init_notebook_mode
from heapq import nlargest
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering

#parâmetros para rodar a biblioteca plotly ofline
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
plotly.offline.init_notebook_mode(connected=True)

#ignorar avisos do código
warnings.filterwarnings('ignore')


In [3]:
#carregar o arquivo de dados na váriavel df_analysis
path = 'mencoes.csv'
df_analysis =  pd.read_csv(path, sep =';')
df_analysis.head()

,Id,Id_materia,SS_count,SS_percent,MS_count,MS_percent,MM_count,MM_percent,MI_count,MI_percent,...,TRANCA_percent,Periodo_ano,Periodo_semestre,Turma,SR_count,SR_percent,Disciplina,Departamento,code,Tipo
0,1,1,0,0.00,0,0.00,0,0.00,0,0.00,...,0.0,2009,2,NaN,0,0.00,INTRODUCAO A CIEN,CIC,113913,GR
1,2,1,11,23.40,20,42.55,11,23.40,3,6.38,...,0.0,2009,2,A,1,2.13,INTRODUCAO A CIEN,CIC,113913,GR
2,3,1,15,18.75,19,23.75,20,25.00,7,8.75,...,17.5,2009,2,AA,5,6.25,INTRODUCAO A CIEN,CIC,113913,GR
3,4,1,20,25.00,27,33.75,15,18.75,4,5.00,...,15.0,2009,2,BB,2,2.50,INTRODUCAO A CIEN,CIC,113913,GR
4,5,1,23,41.07,21,37.50,1,1.79,0,0.00,...,0.0,2009,2,C,9,16.07,INTRODUCAO A CIEN,CIC,113913,GR


In [4]:
#tipos de dados de cada coluna
df_analysis.dtypes

Id                    int64
Id_materia            int64
SS_count              int64
SS_percent          float64
MS_count              int64
MS_percent          float64
MM_count              int64
MM_percent          float64
MI_count              int64
MI_percent          float64
II_count              int64
II_percent          float64
TR_count              int64
TR_percent          float64
TJ_count              int64
TJ_percent          float64
CC_count              int64
CC_percent          float64
DP_count              int64
DP_percent          float64
AP_count              int64
AP_percent          float64
RP_count              int64
RP_percent          float64
APROVA_count          int64
APROVA_percent      float64
REPROVA_count         int64
REPROVA_percent     float64
TRANCA_count          int64
TRANCA_percent      float64
Periodo_ano           int64
Periodo_semestre      int64
Turma                object
SR_count              int64
SR_percent          float64
Disciplina          

In [5]:
# verificar estatpísticas gerais dos dados
df_analysis.describe(include = [np.float64])

,SS_percent,MS_percent,MM_percent,MI_percent,II_percent,TR_percent,TJ_percent,CC_percent,DP_percent,AP_percent,RP_percent,APROVA_percent,REPROVA_percent,TRANCA_percent,SR_percent
count,10501.000000,10501.000000,10501.000000,10501.000000,10501.000000,10501.000000,10501.000000,10501.000000,10501.000000,10501.000000,10501.000000,10501.000000,10501.000000,10501.000000,10501.000000
mean,30.687717,19.808352,15.151670,5.221417,2.337238,2.295199,0.549529,18.361664,0.064182,0.748031,0.000453,66.395869,12.333563,2.844714,4.774417
std,40.393190,27.233021,22.560939,14.705802,7.033920,7.474920,4.280007,38.682527,2.171245,8.535485,0.046451,39.172441,21.926719,8.736680,13.217414
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,39.130000,0.000000,0.000000,0.000000
50%,6.450000,2.220000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,83.870000,0.000000,0.000000,0.000000
75%,62.500000,33.330000,27.420000,3.330000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,17.650000,0.000000,2.700000
max,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,4.760000,100.000000,100.000000,100.000000,100.000000


### <font color=black> Retirada de dados nulos  </font>
### <font color=black> Verificação de quais entradas no dataframe são nulas e para tal realizar a retirada desse valores que podem interferir nas estatísticas.   </font>

In [6]:
#verificadndo entradas nulas do dataframe
print(df_analysis.isnull().sum())

Id                     0
Id_materia             0
SS_count               0
SS_percent             0
MS_count               0
MS_percent             0
MM_count               0
MM_percent             0
MI_count               0
MI_percent             0
II_count               0
II_percent             0
TR_count               0
TR_percent             0
TJ_count               0
TJ_percent             0
CC_count               0
CC_percent             0
DP_count               0
DP_percent             0
AP_count               0
AP_percent             0
RP_count               0
RP_percent             0
APROVA_count           0
APROVA_percent         0
REPROVA_count          0
REPROVA_percent        0
TRANCA_count           0
TRANCA_percent         0
Periodo_ano            0
Periodo_semestre       0
Turma               1882
SR_count               0
SR_percent             0
Disciplina             0
Departamento           0
code                   0
Tipo                   0
dtype: int64


### <font color=black> Verifica-se que as entradas nulas são provenientes do campo turma, isso se dá devido ao fato doe existirem turmas com valores nulos que indicam turmas que não existem que servem para conceder créditos aos estudantes.  </font>
### <font color=black> Devido a isso, essas turmas serão excluidas dessa análise devido ao fato de que essas informações se demonstram como não relevantes para entender o perfil educacional das disciplinas do departamento. </font>

In [7]:
print('Antes:')
print('Old size: %d' % len(df_analysis))
df_analysis_drop = df_analysis.dropna(how = 'any', axis = 'rows')
print('Depois:')
print('New size: %d' % len(df_analysis_drop))
#Verifica-se as quantidades de dados que foram retirados da análise

Antes:
Old size: 10501
Depois:
New size: 8619


### <font color=black> Ao se analisar os valores dos dados, percebeu-se que existem um vasto número de turmas destinadas a disciplinas de conclusão de curso, e as mesmas não refletem bem o fenômenos educacionais que ocorrem nas disciplinas de cada departamento.</font>
### <font color=black> Verificou-se também que existem dados sobre as disciplinas de pós graduação, como essas fogem do escopo do estudo realizado, as mesmas foram retiradas também.</font>

In [8]:
#Filtro de disciplinas apenas colocadas como graduação
df_grad_all = df_analysis_drop[df_analysis_drop.Tipo == 'GR']

In [9]:
print('Antes:')
print('Old size: %d' % len(df_analysis_drop))
print('Depois:')
print('New size: %d' % len(df_grad_all))
#Verifica-se as quantidades de dados que foram retirados da análise

Antes:
Old size: 8619
Depois:
New size: 7173


In [10]:
#Retirada de disciplinas de conclusão de curso 
df_grad = df_grad_all
df_grad = df_grad[df_grad.Disciplina != 'Projeto Final de Graduação']
df_grad = df_grad[df_grad.Disciplina != 'Trabalho Conclusão de']
df_grad = df_grad[df_grad.Disciplina != 'Trabalho de Graduação 1']
df_grad = df_grad[df_grad.Disciplina != 'Trabalho de Graduação 2']
df_grad = df_grad[df_grad.Disciplina != 'PROJETO DE LICENCIATURA']
df_grad = df_grad[df_grad.Disciplina != 'Projeto de Licenciatura 1']
df_grad = df_grad[df_grad.Disciplina != 'Projeto de Licenciatura 2'] 
df_grad = df_grad[df_grad.Disciplina != 'Projeto Final Eng']

In [11]:
print('Antes:')
print('Old size: %d' % len(df_grad_all))
print('Depois:')
print('New size: %d' % len(df_grad))
#Verifica-se as quantidades de dados que foram retirados da análise

Antes:
Old size: 7173
Depois:
New size: 5153


### <font color=black> A retirada de disciplinas de pós graduação reduzem em cerca de 15% dos dados, as quais delimitam o escopo a ser estudado. </font>
### <font color=black> A retirada de disciplinas de conclusão de curso reduzem em cerca de 30% dos dados, o que se é esperado e apesar de reduzir a quantidade dos dados, certamente facilita no entendimento dos fenômenos educacionais que é a proposta feita.</font>

In [12]:

df_ene = df_grad[df_grad.Departamento == 'ENE']
df_cic = df_grad[df_grad.Departamento == 'CIC']

#Dataframes sem a rettirada das disciplinas de conclusão de curso
#df_ene = df_grad_all[df_grad_all.Departamento == 'ENE']
#df_cic = df_grad_all[df_grad_all.Departamento == 'CIC']

In [13]:
#verificar estatísticas do ENE
df_ene.describe(include = [np.float64])

,SS_percent,MS_percent,MM_percent,MI_percent,II_percent,TR_percent,TJ_percent,CC_percent,DP_percent,AP_percent,RP_percent,APROVA_percent,REPROVA_percent,TRANCA_percent,SR_percent
count,2419.000000,2419.000000,2419.000000,2419.000000,2419.000000,2419.000000,2419.000000,2419.000000,2419.0,2419.0,2419.0,2419.000000,2419.000000,2419.000000,2419.000000
mean,12.697272,33.053948,32.651914,8.134845,4.076217,4.001521,1.082580,0.088694,0.0,0.0,0.0,78.403237,16.423989,5.084076,4.212848
std,19.044135,22.533762,21.497412,12.203493,8.041733,8.437755,4.424046,2.883288,0.0,0.0,0.0,21.266396,18.600173,9.613833,8.459057
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
25%,0.000000,15.660000,16.670000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,66.670000,0.000000,0.000000,0.000000
50%,5.560000,30.770000,32.000000,2.630000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,84.000000,10.530000,0.000000,0.000000
75%,16.670000,47.620000,47.060000,12.735000,5.000000,5.260000,0.000000,0.000000,0.0,0.0,0.0,96.000000,26.445000,6.670000,5.560000
max,100.000000,100.000000,100.000000,100.000000,70.000000,100.000000,64.710000,100.000000,0.0,0.0,0.0,100.000000,100.000000,100.000000,100.000000


In [14]:
#verificar estatísticas do CIC
df_cic.describe(include = [np.float64])

,SS_percent,MS_percent,MM_percent,MI_percent,II_percent,TR_percent,TJ_percent,CC_percent,DP_percent,AP_percent,RP_percent,APROVA_percent,REPROVA_percent,TRANCA_percent,SR_percent
count,2734.000000,2734.000000,2734.000000,2734.000000,2734.000000,2734.000000,2734.000000,2734.00000,2734.0,2734.000000,2734.0,2734.000000,2734.000000,2734.000000,2734.000000
mean,32.066357,24.215157,20.602747,5.774382,4.354455,4.130121,0.723467,0.12500,0.0,0.073153,0.0,76.957553,18.063859,4.853552,7.934989
std,38.455441,25.722253,22.933598,11.472993,8.631267,8.564763,4.121540,3.32817,0.0,2.704184,0.0,24.802077,21.566039,9.643328,13.884325
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,61.042500,0.000000,0.000000,0.000000
50%,12.500000,19.085000,16.670000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.0,83.330000,11.110000,0.000000,0.000000
75%,55.970000,35.710000,33.330000,8.330000,5.710000,4.880000,0.000000,0.00000,0.0,0.000000,0.0,100.000000,30.430000,5.880000,11.110000
max,100.000000,100.000000,100.000000,100.000000,72.580000,100.000000,100.000000,100.00000,0.0,100.000000,0.0,100.000000,100.000000,100.000000,100.000000


### <font color=black> Distribuição de Histograms de Aprovações, Reprovações e Departamentos dos Departamentos </font>

In [15]:
#plotagem dos dados de aprovação de ambos os departamentos
df_ene.APROVA_percent.iplot(kind='hist',xTitle='Aprovação %',
                  yTitle='Quantidade', title='Histograma Aprovação ENE')
df_cic.APROVA_percent.iplot(kind='hist',xTitle='Aprovação %',
                  yTitle='Quantidade', title='Histograma Aprovação CIC')

In [16]:
#plotagem dos dados de reprovação de ambos os departamentos
df_ene.REPROVA_percent.iplot(kind='hist',xTitle='Aprovação %',
                  yTitle='Quantidade', title='Histograma Reprovação ENE')
df_cic.REPROVA_percent.iplot(kind='hist',xTitle='Aprovação %',
                  yTitle='Quantidade', title='Histograma Reprovação CIC')

In [17]:
#plotagem dos dados de reprovação de ambos os departamentos
df_ene.TRANCA_percent.iplot(kind='hist',xTitle='Aprovação %',
                  yTitle='Quantidade', title='Histograma Trancamentos ENE')
df_cic.TRANCA_percent.iplot(kind='hist',xTitle='Aprovação %',
                  yTitle='Quantidade', title='Histograma Trancamentos CIC')


In [31]:
def indices (titulo, _type,df):
    df_dpto = df
    top10=[]
    data = []
    for i in range(1,max(df_dpto.Id_materia)):
        total = sum(df_dpto.REPROVA_count[df_dpto.Id_materia == i] + df_dpto.APROVA_count[df_dpto.Id_materia == i] + df_dpto.TRANCA_count[df_dpto.Id_materia == i])
        if(total > 300):
            data.append(sum(_type[df_dpto.Id_materia == i])/total)
        else:
            data.append(0)

    lista = data
    data = sorted( [(x,i) for (i,x) in enumerate(data)], reverse=True )[:10]
    bars=[]
    height=[]
    
    for i in range (0, 10):
        nome = str((df_dpto.Disciplina[df_dpto.Id_materia==data[i][1]+1]).head(1))
#         nome = re.search('    (.*)\nName:',str((df_dpto.Disciplina[df_dpto.Id_materia==data[i][1]+1]).head(1))).group(1)
        bars.append(nome)
        height.append(data[i][0])
        if(titulo=="Reprovação"):
               top10.append(data[i][1]+1)

    height = [x * 100 for x in height]
    data = [go.Bar(
            x= height,
            y= bars,
            marker=dict(
                color='rgb(75, 0, 130)',
                line=dict(
                    color='rgb(75, 0, 130)',
                    width=1),
                    ),
            name= titulo ,
            orientation='h',
            )]
    
    layout = go.Layout(
    xaxis=dict(
        showgrid=True,
        showline=False,
        showticklabels=True,
        zeroline=False,
        title=titulo + ' (%)',
        titlefont=dict(
            size=16,
        ),
        domain=[0.15, 1],
        
        #tickfont=dict(
         #   size=12,
          #  color='rgb(0, 0, 0)',
        #),
        
        
    ),
    yaxis=dict(
        showgrid=True,
        showline=False,
        showticklabels=False,
        zeroline=False,
        title='Disciplinas',
        titlefont=dict(
            size=16,
        ),
        #tickangle=30,
        tickfont=dict(
            size=16,
          #  color='rgb(0, 0, 0)',
        ),
    ),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    title= 'Disiplinas com maiores índice de ' + titulo.lower() 
    )
    a = 0
    annotations = []
    for yd,xd in zip(height,bars):
        if a == 0:
            b = yd
            a = 1

        annotations.append(dict(xref='x1', yref='y1',
                                y=xd, x= b/3,               
                                text=xd,
                                font=dict(family='Times New Roman', size=12,
                                          color='rgb(255, 255, 255)'),
                                showarrow=False))
        annotations.append(dict(xref='x1', yref='y1',
                                y=xd, x= yd + b/15,               
                                text='%.2f'%yd + '%',
                                font=dict(family='Times New Roman', size=12,
                                          color='rgb(0, 0, 0)'),
                                showarrow=False))
 
    
    layout['annotations'] = annotations
    fig = go.Figure(data=data, layout=layout)
    
    plotly.offline.iplot(fig, filename= titulo)

    return lista,top10







In [19]:
def check_course(title, _type,df_dpto, code):
    this_course = df_dpto[df_dpto.code == code]
    total = sum(df_dpto.REPROVA_count[df_dpto.code == code] + df_dpto.APROVA_count[df_dpto.code == code] + df_dpto.TRANCA_count[df_dpto.code == code])
    print(title + ' ' + str(sum(_type[df_dpto.code == code])/total))

code=117536
check_course("Reprovação", df_cic.REPROVA_count ,df_cic, code)
check_course("Trancamento",df_cic.TRANCA_count,df_cic, code)
check_course("Aprovação",df_cic.APROVA_count,df_cic, code)

Reprovação 0.15268225584594222
Trancamento 0.039889958734525444
Aprovação 0.8074277854195323


### <font color=black> Lista das disciplinas que mais reprovam/aprovam/trancam no CIC</font>

In [33]:
reprova_list_cic,top10_cic = indices("Reprovação", df_cic.REPROVA_count ,df_cic)
tranca_list = indices("Trancamento",df_cic.TRANCA_count,df_cic)[0]
aprova_list = indices("Aprovação",df_cic.APROVA_count,df_cic)[0]

### <font color=black> Lista das disciplinas que mais reprovam/aprovam/trancam no ENE</font>

In [34]:
[reprova_list_ene,top10_ene] = indices("Reprovação", df_ene.REPROVA_count,df_ene)
tranca_list = indices("Trancamento",df_ene.TRANCA_count,df_ene)[0]
aprova_list = indices("Aprovação",df_ene.APROVA_count,df_ene)[0]

In [22]:
def timeline (title, id_materia):
    data = []
    xlabel = []
    _this = df_analysis[df_analysis.Id_materia == id_materia]

    for ano in range (min(df_analysis.Periodo_ano), max(df_analysis.Periodo_ano)+1):
        this_year = _this[df_analysis.Periodo_ano == ano]
        for semestre in range (min(df_analysis.Periodo_semestre), max(df_analysis.Periodo_semestre)+1):
            this_semester = this_year[df_analysis.Periodo_semestre == semestre]
            soma = sum(this_semester.REPROVA_count) + sum(this_semester.APROVA_count) + sum(this_semester.TRANCA_count)
            if(soma):
                data.append(sum(this_semester.REPROVA_count)/soma)
                xlabel.append(str(ano) + '/' + str(semestre))
    
    aux = re.search('    (.*)\nName:',str((df_analysis.Disciplina[df_analysis.Id_materia==id_materia]).head(1))).group(1)
    data = [x * 100 for x in data]
    trace = go.Scatter(
        x = xlabel,
        y = data,
        mode = 'lines+markers',
        name = 'lines+markers'
        )
    
    data2 = [trace]
    
    layout = go.Layout(
    title= aux,
    xaxis=dict(
        title='Período',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Reprovação (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    )
    fig = go.Figure(data=data2, layout=layout)
    
    
    plotly.offline.iplot(fig, filename='line-mode')




#timeline('REPROVAÇÃO: ', 202) # Eletromagnetismo
#timeline('REPROVAÇÃO: ', 355) # Princom
#timeline('REPROVAÇÃO: ', 426) # SD
#timeline('REPROVAÇÃO: ', 229) # SD




In [38]:
index = 1
top10 = top10_cic
for top in top10:
    timeline('', top)
    index+=1

### <font color=black> Aqui foi criada uma lista das disciplinas que apresentam maiores valores para o desvio padrão em dois semestre consecultivos, para tentar entender os fenômenos que levaram a esse acontecimento.</font>

In [24]:
def lista_devpad(depart):
    
    if depart == 'CIC':
        df_dpto = df_cic
    if depart == 'ENE':
        df_dpto = df_ene
    variacao_df = []
    ano_df = []
    sem_df = []
    dis_df = []


    for materia in df_dpto.Disciplina.unique():
        if (materia == 'Projeto Final de Graduação' or materia == 'Trabalho Conclusão de'):
            continue
        ap = df_dpto.APROVA_count[df_dpto['Disciplina'] == materia]
        re = df_dpto.REPROVA_count[df_dpto['Disciplina'] == materia]
        tr = df_dpto.TRANCA_count[df_dpto['Disciplina'] == materia]
        soma = sum(ap+re+tr)
        if soma > 300:
            for ano in df_dpto.Periodo_ano[df_dpto['Periodo_ano'] > 2015].unique():
                for sem in df_dpto.Periodo_semestre.unique():
                    df_temp = (df_dpto[((df_dpto['Periodo_ano'] == ano) & (df_dpto['Periodo_semestre'] == sem))])
                    std = df_temp.APROVA_percent[df_temp['Disciplina'] == materia].std()
                    if std > 0:
                        variacao_df.append(std)
                        ano_df.append(ano)
                        sem_df.append(sem)
                        dis_df.append(materia)

    val = nlargest(10,variacao_df)
    for i in val:
        temp = variacao_df.index(i)
        print(dis_df[temp],ano_df[temp],sem_df[temp],variacao_df[temp])
    return

In [25]:
print('ENE')
lista_devpad('ENE')
print('CIC')
lista_devpad('CIC')

ENE
TOPICOS EM ENGENHARIA 2018 1 61.87184335382291
Top em Redes de 2016 2 29.465139572043437
Circuitos Elétricos 2016 2 28.734328482380327
SISTEMAS DE COMUNICACOES 1 2018 1 26.382154006070095
Princípios de Comunicação 2016 1 23.977990950035824
Circuitos Elétricos 2017 1 22.807091733347622
SISTEMAS DIGITAIS 2 2016 1 21.72939138586261
Sistemas Digitais 2016 1 20.61981894521223
Top em Redes de 2017 2 20.51907405318281
Circuitos Elétricos 2016 1 20.040073186825776
CIC
INTRODUCAO A 2016 1 62.22539674441618
LINGUAGENS DE PROGRAMACAO 2017 1 39.68990362800091
ESTUDOS SISTEMAS 2016 1 31.132471299766248
LINGUAGENS DE PROGRAMACAO 2017 2 29.537654499525402
ORG ARQ DE COMPUTADORES 2017 1 28.659676085631766
Tecnicas de Programação 1 2017 2 28.443122777454192
ESTUDOS SISTEMAS 2016 2 27.513788321534307
ORG ARQ DE COMPUTADORES 2017 2 26.77652143203071
ENGENHARIA DE SOFTWARE 2018 1 25.894250327051367
Algoritmos Progr 2016 2 25.408983788078913


In [26]:
ene_grad = [sum(df_ene.SS_count), sum(df_ene.MS_count), sum(df_ene.MM_count), sum(df_ene.MI_count), sum(df_ene.II_count), sum(df_ene.II_count)]
ene_grad = np.divide(ene_grad,sum(ene_grad))
cic_grad = [sum(df_cic.SS_count), sum(df_cic.MS_count), sum(df_cic.MM_count), sum(df_cic.MI_count), sum(df_cic.II_count), sum(df_cic.II_count)]
cic_grad = np.divide(cic_grad,sum(cic_grad))

df_ene_tcc = df_analysis[df_analysis.Departamento == 'ENE']
df_cic_tcc = df_analysis[df_analysis.Departamento == 'CIC']

ene_grad_tcc = [sum(df_ene_tcc.SS_count), sum(df_ene_tcc.MS_count), sum(df_ene_tcc.MM_count), sum(df_ene_tcc.MI_count), sum(df_ene_tcc.II_count), sum(df_ene_tcc.II_count)]
ene_grad_tcc = np.divide(ene_grad_tcc,sum(ene_grad_tcc))
cic_grad_tcc = [sum(df_cic_tcc.SS_count), sum(df_cic_tcc.MS_count), sum(df_cic_tcc.MM_count), sum(df_cic_tcc.MI_count), sum(df_cic_tcc.II_count), sum(df_cic_tcc.II_count)]
cic_grad_tcc = np.divide(cic_grad_tcc,sum(cic_grad_tcc))

ene_grad = [x * 100 for x in ene_grad]
ene_grad_tcc = [x * 100 for x in ene_grad_tcc]

cic_grad = [x * 100 for x in cic_grad]
cic_grad_tcc = [x * 100 for x in cic_grad_tcc]





### <font color=black> A partir dos dados das menções das disciplinas foi tentado traçar relações entre esses dados.</font>

In [27]:
trace = go.Bar(
    x= ['SS', 'MS', 'MM', 'MI', 'II', 'SR'],
    y= ene_grad_tcc,
    marker=dict(
        color='rgb(26, 118, 255)'
    )
)
layout = go.Layout(
    title='Menções ENE',
    xaxis=dict(
        title='Menção',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
     yaxis=dict(
        title='Ocorrênica (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    )
    

data = [trace]
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='style-bar')


trace = go.Bar(
    x= ['SS', 'MS', 'MM', 'MI', 'II', 'SR'],
    y= cic_grad_tcc,
    marker=dict(
        color='rgb(26, 118, 255)'
    )
)
layout = go.Layout(
    title='Menções CIC',
    xaxis=dict(
        title='Menção',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
     yaxis=dict(
        title='Ocorrênica (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    )
    

data = [trace]
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='style-bar')

### <font color=black> Como citado antes, segue aqui a defasagem dos dados causados pelas disciplinas de conclusão de curso, apesar de não serem grandes, elas continuam não trazendo informação relevante ao estudo realizado</font>

In [28]:


trace1 = go.Bar(
    x= ['SS', 'MS', 'MM', 'MI', 'II', 'SR'],
    y= ene_grad,
    name='Sem TCC',
    marker=dict(
        color='rgb(55, 83, 109)'
    )
)
trace2 = go.Bar(
    x= ['SS', 'MS', 'MM', 'MI', 'II', 'SR'],
    y= ene_grad_tcc,
    name='Com TCC',
    marker=dict(
        color='rgb(26, 118, 255)'
    )
)

data = [trace1, trace2]

layout = go.Layout(
    title='Comparativo de Menções (ENE)',
    xaxis=dict(
        title='Menção',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Ocorrênica (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='style-bar')


trace1 = go.Bar(
    x= ['SS', 'MS', 'MM', 'MI', 'II', 'SR'],
    y= cic_grad,
    name='Sem TCC',
    marker=dict(
        color='rgb(55, 83, 109)'
    )
)
trace2 = go.Bar(
    x= ['SS', 'MS', 'MM', 'MI', 'II', 'SR'],
    y= cic_grad_tcc,
    name='Com TCC',
    marker=dict(
        color='rgb(26, 118, 255)'
    )
)

data = [trace1, trace2]

layout = go.Layout(
    title='Comparativo de Menções (CIC)',
    xaxis=dict(
        title='Menção',
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Ocorrênica (%)',
        titlefont=dict(
            size=16,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, filename='style-bar')





In [29]:
#Kmeans analysis
cluster_map = []
x=np.reshape(reprova_list_ene, (-1, 1))
clusters = 3
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(x)
easy = []; medium = []; hard = []
x = [j * 100 for j in x]
x = [item for sublist in x for item in sublist]

cluster_map = {}
cluster_map['cluster'] = kmeans.labels_
cluster_map_values_x = x
cluster_map_values_y = np.zeros_like(reprova_list_ene)

trace1 = {
  "x": cluster_map_values_x[cluster_map['cluster'] == 1].tolist(), 
  "y": cluster_map_values_y[cluster_map['cluster'] == 1].tolist(), 
  "marker": {
    "color": "rgb(228,26,28)", 
    "line": {"width": 1}, 
    "opacity": 1, 
    "size": 14
  }, 
  "mode": "markers", 
  "name": "1", 
  "text": "1", 
  "type": "scatter", 
  "uid": "0cd8fa1c-dc0d-11e8-ab78-d89ef30f304d"
}
trace2 = {
  "x": cluster_map_values_x[cluster_map['cluster'] == 2].tolist(), 
  "y": cluster_map_values_y[cluster_map['cluster'] == 2].tolist(), 
  "marker": {
    "color": "rgb(55,126,184)", 
    "line": {"width": 1}, 
    "opacity": 1, 
    "size": 14
  }, 
  "mode": "markers", 
  "name": "2", 
  "text": "2", 
  "type": "scatter", 
  "uid": "0cd8fa1d-dc0d-11e8-9bef-d89ef30f304d"
}
trace3 = {
  "x": cluster_map_values_x[cluster_map['cluster'] == 0].tolist(), 
  "y": cluster_map_values_y[cluster_map['cluster'] == 0].tolist(), 
  "marker": {
    "color": "rgb(77,175,74)", 
    "line": {"width": 1}, 
    "opacity": 1, 
    "size": 14
  }, 
  "mode": "markers", 
  "name": "3", 
  "text": "3", 
  "type": "scatter", 
  "uid": "0cd8fa1e-dc0d-11e8-a270-d89ef30f304d"
}


data = go.Data([trace1, trace2, trace3])
layout = go.Layout(
    xaxis = dict( 
        zeroline = False,
        title = 'Reprovação (%)',
        tickfont=dict(
            size=14,
        )
    ), 
    yaxis = dict( 
        zeroline =  False,
        #title = 'Reprovação (%)',
        tickfont=dict(
            size=14,
        )
    ),   
  title = "Aglomeração de Disiplinas por Reprovação"
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

        

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
#Kmeans analysis 
x=np.reshape(reprova_list_cic, (-1, 1))
clusters = 4
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(x)
easy = []; medium = []; hard = []
x = [j * 100 for j in x]
x = [item for sublist in x for item in sublist]
cluster_map['cluster'] = kmeans.labels_
cluster_map['values_x'] = x
cluster_map['values_y'] = np.zeros_like(reprova_list_cic)

trace1 = {
  "x": cluster_map.values_x[cluster_map['cluster'] == 1].tolist(), 
  "y": cluster_map.values_y[cluster_map['cluster'] == 1].tolist(), 
  "marker": {
    "color": "rgb(228,26,28)", 
    "line": {"width": 1}, 
    "opacity": 1, 
    "size": 14
  }, 
  "mode": "markers", 
  "name": "1", 
  "text": "1", 
  "type": "scatter", 
  "uid": "0cd8fa1c-dc0d-11e8-ab78-d89ef30f304d"
}
trace2 = {
  "x": cluster_map.values_x[cluster_map['cluster'] == 3].tolist(), 
  "y": cluster_map.values_y[cluster_map['cluster'] == 3].tolist(), 
  "marker": {
    "color": "rgb(55,126,184)", 
    "line": {"width": 1}, 
    "opacity": 1, 
    "size": 14
  }, 
  "mode": "markers", 
  "name": "2", 
  "text": "2", 
  "type": "scatter", 
  "uid": "0cd8fa1d-dc0d-11e8-9bef-d89ef30f304d"
}
trace3 = {
  "x": cluster_map.values_x[cluster_map['cluster'] == 0].tolist(), 
  "y": cluster_map.values_y[cluster_map['cluster'] == 0].tolist(), 
  "marker": {
    "color": "rgb(77,175,74)", 
    "line": {"width": 1}, 
    "opacity": 1, 
    "size": 14
  }, 
  "mode": "markers", 
  "name": "3", 
  "text": "3", 
  "type": "scatter", 
  "uid": "0cd8fa1e-dc0d-11e8-a270-d89ef30f304d"
}
trace4 = {
  "x": cluster_map.values_x[cluster_map['cluster'] == 2].tolist(), 
  "y": cluster_map.values_y[cluster_map['cluster'] == 2].tolist(), 
  "marker": {
    "color": "rgb(126,77,228)", 
    "line": {"width": 1}, 
    "opacity": 1, 
    "size": 14
  }, 
  "mode": "markers", 
  "name": "4", 
  "text": "4", 
  "type": "scatter", 
  "uid": "0cd8fa1f-dc0d-11e8-a842-d89ef30f304d"
}

data = go.Data([trace1, trace2, trace3, trace4])
layout = go.Layout(
    xaxis = dict( 
        zeroline = False,
        title = 'Reprovação (%)',
        tickfont=dict(
            size=14,
        )
    ), 
    yaxis = dict( 
        zeroline =  False,
        #title = 'Reprovação (%)',
        tickfont=dict(
            size=14,
        )
    ),   
  title = "Aglomeração de Disiplinas por Reprovação"
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

        

In [ ]:
#Kmeans analysis 
x=np.reshape(reprova_list, (-1, 1))
clusters = 5
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(x)
easy = []; medium = []; hard = []
x = [j * 100 for j in x]
x = [item for sublist in x for item in sublist]
cluster_map['cluster'] = kmeans.labels_
cluster_map['values_x'] = x
cluster_map['values_y'] = np.zeros_like(reprova_list)

trace1 = {
  "x": cluster_map.values_x[cluster_map['cluster'] == 1].tolist(), 
  "y": cluster_map.values_y[cluster_map['cluster'] == 1].tolist(), 
  "marker": {
    "color": "rgb(228,26,28)", 
    "line": {"width": 1}, 
    "opacity": 1, 
    "size": 14
  }, 
  "mode": "markers", 
  "name": "1", 
  "text": "1", 
  "type": "scatter", 
  "uid": "0cd8fa1c-dc0d-11e8-ab78-d89ef30f304d"
}
trace2 = {
  "x": cluster_map.values_x[cluster_map['cluster'] == 4].tolist(), 
  "y": cluster_map.values_y[cluster_map['cluster'] == 4].tolist(), 
  "marker": {
    "color": "rgb(55,126,184)", 
    "line": {"width": 1}, 
    "opacity": 1, 
    "size": 14
  }, 
  "mode": "markers", 
  "name": "2", 
  "text": "2", 
  "type": "scatter", 
  "uid": "0cd8fa1d-dc0d-11e8-9bef-d89ef30f304d"
}
trace3 = {
  "x": cluster_map.values_x[cluster_map['cluster'] == 2].tolist(), 
  "y": cluster_map.values_y[cluster_map['cluster'] == 2].tolist(), 
  "marker": {
    "color": "rgb(77,175,74)", 
    "line": {"width": 1}, 
    "opacity": 1, 
    "size": 14
  }, 
  "mode": "markers", 
  "name": "3", 
  "text": "3", 
  "type": "scatter", 
  "uid": "0cd8fa1e-dc0d-11e8-a270-d89ef30f304d"
}
trace4 = {
  "x": cluster_map.values_x[cluster_map['cluster'] == 0].tolist(), 
  "y": cluster_map.values_y[cluster_map['cluster'] == 0].tolist(), 
  "marker": {
    "color": "rgb(126,77,228)", 
    "line": {"width": 1}, 
    "opacity": 1, 
    "size": 14
  }, 
  "mode": "markers", 
  "name": "4", 
  "text": "4", 
  "type": "scatter", 
  "uid": "0cd8fa1f-dc0d-11e8-a842-d89ef30f304d"
}
trace5 = {
  "x": cluster_map.values_x[cluster_map['cluster'] == 3].tolist(), 
  "y": cluster_map.values_y[cluster_map['cluster'] == 3].tolist(), 
  "marker": {
    "color": "rgb(180,225,50)", 
    "line": {"width": 1}, 
    "opacity": 1, 
    "size": 14
  }, 
  "mode": "markers", 
  "name": "5", 
  "text": "5", 
  "type": "scatter", 
  "uid": "0cd8fa20-dc0d-11e8-9761-d89ef30f304d"
}

data = go.Data([trace1, trace2, trace3, trace4,trace5])
layout = go.Layout(
    xaxis = dict( 
        zeroline = False,
        title = 'Reprovação (%)',
        tickfont=dict(
            size=14,
        )
    ), 
    yaxis = dict( 
        zeroline =  False,
        #title = 'Reprovação (%)',
        tickfont=dict(
            size=14,
        )
    ),   
  title = "Aglomeração de Disiplinas por Reprovação"
)
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

        

In [ ]:



trace1 = {
  "x": cluster_map.values_x[cluster_map['cluster'] == 0].tolist(), 
  "y": cluster_map.values_y[cluster_map['cluster'] == 0].tolist(), 
  "marker": {
    "color": "rgb(228,26,28)", 
    "line": {"width": 1}, 
    "opacity": 1, 
    "size": 14
  }, 
  "mode": "markers", 
  "name": "1", 
  "text": "1", 
  "type": "scatter", 
  "uid": "0cd8fa1c-dc0d-11e8-ab78-d89ef30f304d"
}
trace2 = {
  "x": cluster_map.values_x[cluster_map['cluster'] == 1].tolist(), 
  "y": cluster_map.values_y[cluster_map['cluster'] == 1].tolist(), 
  "marker": {
    "color": "rgb(55,126,184)", 
    "line": {"width": 1}, 
    "opacity": 1, 
    "size": 14
  }, 
  "mode": "markers", 
  "name": "0", 
  "text": "0", 
  "type": "scatter", 
  "uid": "0cd8fa1d-dc0d-11e8-9bef-d89ef30f304d"
}
trace3 = {
  "x": cluster_map.values_x[cluster_map['cluster'] == 2].tolist(), 
  "y": cluster_map.values_y[cluster_map['cluster'] == 2].tolist(), 
  "marker": {
    "color": "rgb(77,175,74)", 
    "line": {"width": 1}, 
    "opacity": 1, 
    "size": 14
  }, 
  "mode": "markers", 
  "name": "2", 
  "text": "2", 
  "type": "scatter", 
  "uid": "0cd8fa1e-dc0d-11e8-a270-d89ef30f304d"
}
trace4 = {
  "x": cluster_map.values_x[cluster_map['cluster'] == 3].tolist(), 
  "y": cluster_map.values_y[cluster_map['cluster'] == 3].tolist(), 
  "marker": {
    "color": "rgb(126,77,228)", 
    "line": {"width": 1}, 
    "opacity": 1, 
    "size": 14
  }, 
  "mode": "markers", 
  "name": "3", 
  "text": "3", 
  "type": "scatter", 
  "uid": "0cd8fa1f-dc0d-11e8-a842-d89ef30f304d"
}

data = go.Data([trace1, trace2, trace3, trace4])
layout = {
  "scene": {
    "xaxis": {"zeroline": False}, 
    "yaxis": {"zeroline": False}
  }, 
  "title": "2d point clustering"
}
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [ ]:
df_dpto = df_cic
def indices (titulo, _type):
    data = []
    for i in range(1,max(df_dpto.Id_materia)):
        total = sum(df_dpto.REPROVA_count[df_dpto.Id_materia == i] + df_dpto.APROVA_count[df_dpto.Id_materia == i] + df_dpto.TRANCA_count[df_dpto.Id_materia == i])
        if(total > 300):
            data.append(sum(_type[df_dpto.Id_materia == i])/total)
    
    return data

reprova_list = indices("REPROVAÇÕES", df_dpto.REPROVA_count)
tranca_list = indices("TRANCAMENTOS",df_dpto.TRANCA_count)
aprova_list = indices("APROVAÇÕES",df_dpto.APROVA_count)

f = plt.figure(figsize=(15,3))
ax = f.add_subplot(131)
ax2 = f.add_subplot(132)
ax3 = f.add_subplot(133)
ax.hist(aprova_list); ax.title.set_text('Histograma de Aprovação')
ax2.hist(reprova_list); ax2.title.set_text('Histograma de Reprovação')
ax3.hist(tranca_list); ax3.title.set_text('Histograma de Trancamento')


In [ ]:
plt.scatter(reprova_list,tranca_list);

'''
X = np.array([reprova_list, tranca_list])
kmeans = KMeans(n_clusters=3,random_state=100).fit(X)

plt.plot(reprova_list, tranca_list, '*')
plt.plot(kmeans.cluster_centers_[:,1], kmeans.cluster_centers_[:,2] ,'r*')
plt.show()
'''


In [ ]:
# BOX PLOT

data=np.reshape(reprova_list_ene, (-1, 1));
fig1, ax1 = plt.subplots();
ax1.set_title('Basic Plot');
ax1.boxplot(reprova_list_ene);
print (reprova_list_ene)

In [ ]:
spread = np.random.rand(50) * 100
center = np.ones(25) * 50
flier_high = np.random.rand(10) * 100 + 100
flier_low = np.random.rand(10) * -100
data = np.concatenate((spread, center, flier_high, flier_low))
print(data)
fig1, ax1 = plt.subplots();
ax1.set_title('Basic Plot');
ax1.boxplot(data);